In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

Mounted at /content/drive
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project/spectroconv
     |████████████████████████████████| 349 kB 28.4 MB/s 
     |████████████████████████████████| 6.2 MB 31.3 MB/s 
     |████████████████████████████████| 81 kB 11.6 MB/s 
     |████████████████████████████████| 132 kB 71.0 MB/s 
     |████████████████████████████████| 79 kB 10.3 MB/s 
     |████████████████████████████████| 8.7 MB 68.7 MB/s 
     |████████████████████████████████| 138 kB 73.7 MB/s 
     |████████████████████████████████| 65 kB 4.7 MB/s 
     |████████████████████████████████| 127 kB 76.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [4]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May  2 22:10:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    14W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from models import wb_experiment
import copy

## Set up common model parameters

In [12]:
# https://arxiv.org/pdf/1605.09507.pdf
# https://archives.ismir.net/ismir2019/paper/000074.pdf

n_mels = 128
wb_config = {
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'group': None,
    'name': None,
}

config = {
    'data': {
        'train': {
            'name': f'nsynth-full-{n_mels}-train'
        },
        'val': {
            'name': f'nsynth-full-{n_mels}-val'
        }
    },
    'model': {
        'id': None,
        'name': None
    },
    'logging': {
        'batch_log_interval': 250,
        'eval_every_batch': True,
        'eval_every_epoch': True,
    },
    'parameters': {
      'epochs': 8,
      # Taenzer setting
      'batch_size': 124, 
      'optimizer': 'adam',
      'learning_rate': 0.0001,
      'dropout': 0.25,
      'l2_reg': 0.0000,
      'kernel_size': 3,
      'num_channels': 32,
      'num_conv_layers': 4,
      'linear_layer_size': 1024,
    },
}

# CNN Training + save plot data to wandb

In [13]:
cnn_wb_config = copy.deepcopy(wb_config)
cnn_wb_config['group'] = 'cnn'
cnn_wb_config['name'] = 'Taenzer-CNN'

cnn_config = copy.deepcopy(config)
cnn_config['model']['id'] = wb_experiment.ModelType.Taenzer
cnn_config['model']['name'] = 'Taenzer-CNN'

In [ ]:
trainer = wb_experiment.TrainExperiment(wb_config=wb_config, login_key=True)
trainer.run_train(config, save_model=False)

# LC (local dim = 0) Training + save plot data to wandb

In [ ]:
cnn_wb_config = copy.deepcopy(wb_config)
cnn_wb_config['group'] = 'lc'
cnn_wb_config['name'] = 'Taenzer-LC-0'

cnn_config = copy.deepcopy(config)
cnn_config['model']['id'] = wb_experiment.ModelType.LC
cnn_config['model']['name'] = 'Taenzer-LC-0'
cnn_config['parameters']['local_dim'] = 0
cnn_config['parameters']['num_conv_layers'] = 3 # last layer is replaced by

In [ ]:
trainer = wb_experiment.TrainExperiment(wb_config=wb_config, login_key=True)
trainer.run_train(config, save_model=False)